### Introduction where you discuss the business problem and who would be interested in this project.

In [9]:
!pip install folium
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


## Foursquare credentials

In [10]:
CLIENT_ID = 'JEX1D22PTBBUZWVQY3TOLFZXSKEPA3DCFYKXF4WF5QZ0NQAY' # your Foursquare ID
CLIENT_SECRET = '4MA5N2DB5VACRSSU03JHUWDOCJGNDDL3XEVQUZTDAGET5QN1' # your Foursquare Secret
VERSION = '20200101' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: JEX1D22PTBBUZWVQY3TOLFZXSKEPA3DCFYKXF4WF5QZ0NQAY


### Locating the comic shops

In [11]:
LIMIT = 100 # Maximum is 100
cities = ['Chicago, IL','Milwaukee, WI', 'Madison, WI']
results_comics = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "52f2ab2ebcbc57f1066b8b18") # Comic stores https://developer.foursquare.com/docs/resources/categories
    results_comics[city] = requests.get(url).json()

In [12]:
df_venues={}
for city in cities:
    venues = json_normalize(results_comics[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [40]:
maps = {}
for city in cities:
    city_lat = np.mean([results_comics[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results_comics[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results_comics[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results_comics[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of comic stores in {city} = ", results_comics[city]['response']['totalResults'])

Total number of comic stores in Chicago, IL =  18
Total number of comic stores in Milwaukee, WI =  4
Total number of comic stores in Madison, WI =  4


In [14]:
maps[cities[0]]

In [15]:
maps[cities[1]]

In [16]:
maps[cities[2]]

### Locate the movie theaters

In [17]:
results_movies = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d17f941735") # Movie theaters https://developer.foursquare.com/docs/resources/categories
    results_movies[city] = requests.get(url).json()

In [18]:
df_movievenues={}
for city in cities:
    venues = json_normalize(results_movies[city]['response']['groups'][0]['items'])
    df_movievenues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_movievenues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [41]:
movie_maps = {}
for city in cities:
    movie_city_lat = np.mean([results_movies[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results_movies[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    movie_city_lng = np.mean([results_movies[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results_movies[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    movie_maps[city] = folium.Map(location=[movie_city_lat, movie_city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_movievenues[city]['Lat'], df_movievenues[city]['Lng'], df_movievenues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='Red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(movie_maps[city])  
    print(f"Total number of movie theaters in {city} = ", results_movies[city]['response']['totalResults'])

Total number of movie theaters in Chicago, IL =  85
Total number of movie theaters in Milwaukee, WI =  10
Total number of movie theaters in Madison, WI =  17


In [20]:
movie_maps[cities[0]]

In [21]:
movie_maps[cities[1]]

In [22]:
movie_maps[cities[2]]

### What is the mean distance of the comic shops from the mean coordinate of the theaters in the city?

In [75]:


for city in cities:
    movie_city_lat = np.mean([results_movies[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results_movies[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    movie_city_lng = np.mean([results_movies[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results_movies[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    movie_maps[city] = folium.Map(location=[movie_city_lat, movie_city_lng], zoom_start=11)
    movie_venue_mean_coor = [df_movievenues[city]['Lat'].mean(), df_movievenues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([movie_venue_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        movie_venue_mean_coor,
        radius=10,
        popup=label,
        color='Blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])    

    
    print(city)
    Matrix = np.apply_along_axis(lambda x: np.linalg.norm(x - movie_venue_mean_coor),1,df_venues[city][['Lat','Lng']].values)
   
    print("The Mean distance of the comic book shops to the mean coordinate for the movie theaters is", np.mean(Matrix))
    print("The closest comic book shops is ", np.amin(Matrix), " away.")
    

Chicago, IL
The Mean distance of the comic book shops to the mean coordinate for the movie theaters is 0.09545175878276443
The closest comic book shops is  0.02614300688037464  away.
Milwaukee, WI
The Mean distance of the comic book shops to the mean coordinate for the movie theaters is 0.04999490929035806
The closest comic book shops is  0.03183524650700028  away.
Madison, WI
The Mean distance of the comic book shops to the mean coordinate for the movie theaters is 0.06281748553155962
The closest comic book shops is  0.02611399863260902  away.


In [76]:
maps[cities[0]]

In [77]:
maps[cities[1]]

In [78]:
maps[cities[2]]